In [1]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# goals
# - line history
# - % bets
# - % money
# NFL side and total

In [6]:
# scraping function
def scrape_for_soup(base_url, url):
    # send GET request
    response = requests.get(base_url+url)

    # status check
    if response.status_code != 200:
        return(f"Error: {response.status_code}")

    # parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [20]:
base_url = 'https://www.sportsbookreview.com'
url = '/betting-odds/nfl-football/'
soup = scrape_for_soup(base_url, url)

# parse links for game spread and spread history
games = soup.find_all('div', id='tbody-nfl')
main_links = set()
line_links = set()
for game in games:
    for a_tag in game.find_all('a', href=True):
        link = a_tag['href']
        if 'scores' in link:
            main_links.add(link)
        if 'line-history' in link:
            line_links.add(link)

In [21]:
main_links

{'/scores/nfl-football/matchup/305024/',
 '/scores/nfl-football/matchup/305026/',
 '/scores/nfl-football/matchup/305028/',
 '/scores/nfl-football/matchup/305031/',
 '/scores/nfl-football/matchup/305034/',
 '/scores/nfl-football/matchup/305037/',
 '/scores/nfl-football/matchup/305040/',
 '/scores/nfl-football/matchup/305042/',
 '/scores/nfl-football/matchup/305044/',
 '/scores/nfl-football/matchup/305047/',
 '/scores/nfl-football/matchup/305049/',
 '/scores/nfl-football/matchup/305051/',
 '/scores/nfl-football/matchup/305052/',
 '/scores/nfl-football/matchup/305053/',
 '/scores/nfl-football/matchup/305059/',
 '/scores/nfl-football/matchup/305067/'}

In [9]:
len(main_links)

16

In [ ]:
# next step - iterate through each link to scrape target data

In [22]:
# next step - iterate through each link to scrape target data
# send request
game_url = list(main_links)[0]
soup = scrape_for_soup(base_url, game_url)
# pull number bets on spread and o/u

In [24]:
soup

<!DOCTYPE html>
<html lang="en-us"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Denver Broncos vs New York Jets Matchup - Sunday, September 29, 2024 - 1:00 PM EDT</title><meta content="Compare Denver Broncos vs New York Jets matchups. Up-to-date betting odds of over 30+ sportsbooks available including money lines, point spreads, totals and futures." name="description"/><meta content="website" property="og:type"/><meta content="Denver Broncos vs New York Jets Matchup - Sunday, September 29, 2024 - 1:00 PM EDT" property="og:title"/><meta content="" property="og:description"/><meta content="https://www.sportsbookreview.com/scores/nfl-football/matchup/305067/" property="og:url"/><meta content="https://img.sportsbookreview.com/images/sbr-open-graph.png" property="og:image"/><meta content="The SBR logo with the bull at the top of the SBR letters" property="og:image:alt"/><meta content="Sportsbook Review" property="og:site_name"/><meta content="summa